In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import albumentations as A
from PIL import Image
import pickle
import numpy as np
import pandas as pd
import os
import math

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cpu


# Architecture Definition

In [25]:
# tuple: (filters, size, stride), single convolutional layer

# ["residual", repetitions]: residual layers, let prev_channel be the number of channels of the previous layer,
# the sequence of conv with filters of size: prev_channels//2 ---> prev_channels ---> residual connection
# is repeated for "repetitions" times

# ["residualYolo", repetitions]: same as "residual" but some feature maps are saved to be used with the YOLO network

# ["avgpool"]: avearge pooling with output of size 1x1 for each channel

# ["prediction"]: last convolutional layer which produces a number of channels equal to the number of classes


darknet_architecture = [
    (32, 3, 1),
    (64, 3, 2),
    ["residual", 1],
    (128, 3, 2),
    ["residual", 2],
    (256, 3, 2),
    ["residualYolo", 8],
    (512, 3, 2),
    ["residualYolo", 8],
    (1024, 3, 2),
    ["residual", 4],
    ["avgpool"],
    ["prediction"]
]

# Blocks Definition

In [12]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bn_act=True, **kwargs):
        super().__init__()
        padding=1 if kernel_size == 3 else 0
        self.conv = nn.Conv2d(in_channels,
                              out_channels,
                              kernel_size=kernel_size,
                              bias=not bn_act,
                              padding=padding, 
                              **kwargs)
        # if batchnorm, then leaky relu as activation function
        self.use_bn_act = bn_act
        if self.use_bn_act:
            self.bn = nn.BatchNorm2d(out_channels)
            self.leaky = nn.LeakyReLU(0.1)

    def forward(self, x):
        if self.use_bn_act:
            return self.leaky(self.bn(self.conv(x)))
        else:
            return self.conv(x)

In [13]:
class ConvBlock(nn.Module):
    def __init__(self, channels, use_residual=True, num_repeats=1):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(num_repeats):
            self.layers += [
                nn.Sequential(
                    ConvLayer(channels, channels // 2, kernel_size=1),
                    ConvLayer(channels // 2, channels, kernel_size=3),
                )
            ]

        self.use_residual = use_residual
        self.num_repeats = num_repeats

    def forward(self, x):
        for layer in self.layers:
            x = layer(x) + self.use_residual * x

        return x

In [14]:
# Block of convolutional layers with feature map to be saved for detections
class ConvBlockYolo(nn.Module):
    def __init__(self, channels, num_repeats=1):
        super().__init__()
        self.layers = nn.ModuleList()
        self.feat_map = None
        num_layer = 1
        for i in range(num_repeats):
            if num_layer == 5:
                self.layers += [
                    ConvLayer(channels, channels // 2, kernel_size=1),
                    ConvLayer(channels // 2, channels, kernel_size=3)
                ]
            else:
                self.layers += [
                    nn.Sequential(
                        ConvLayer(channels, channels // 2, kernel_size=1),
                        ConvLayer(channels // 2, channels, kernel_size=3),
                    )
                ]
            num_layer += 2

    def forward(self, x):
        self.feat_map = None
        for layer in self.layers:
            if isinstance(layer, ConvLayer):
                if self.feat_map == None:
                    self.feat_map = layer(x)
                else:
                    x = layer(self.feat_map) + x
            else:
                x = layer(x) + x

        return x

# Network Definition

In [26]:
class darknet53(nn.Module):
    def __init__(self, architecture, num_classes, in_channels=3):
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.layers = self._create_layers(architecture)
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.reshape(x.shape[0], x.shape[1])
        return x
        
    def _create_layers(self, architecture):
        layers = nn.ModuleList()
        in_channels = self.in_channels
        for module in architecture:
            # conv layer
            if isinstance(module, tuple):
                if module[-1] == "linear":
                    bn_act = False
                else:
                    bn_act = True
                layers.append(ConvLayer(in_channels, 
                                        module[0], 
                                        kernel_size=module[1],
                                        bn_act=bn_act,
                                        stride=module[2])
                             )
                in_channels = module[0]
                continue
            
            if isinstance(module, list):
                # residual block
                if module[0] == "residual":
                    layers.append(ConvBlock(in_channels,
                                            num_repeats=module[1])
                                 )
                    continue
                    
                # residual block with feature map to be saved for detection
                elif module[0] == "residualYolo":
                    layers.append(ConvBlockYolo(in_channels,
                                            num_repeats=module[1])
                                 )
                
                # average pool
                elif module[0] == "avgpool":
                    layers.append(nn.AdaptiveAvgPool2d(1))
                    continue
                
                # last convolutional layer
                elif module[0] == "prediction":
                    layers.append(ConvLayer(in_channels,
                                            self.num_classes,
                                            kernel_size=1,
                                            bn_act=False,
                                            stride=1)
                                 )
                    continue
                    
        return layers

# Dataset

In [8]:
class imgnet_trainset(Dataset):
    def __init__(self, df, path="./ImageNet_darknet_dataset/data/images"):
        self.transform = transforms.PILToTensor()
        self.df = df
        self.path = path
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.path, row["filename"].split("_")[0], row["filename"]))
        return (self.transform(img).float()/255, row["label"])
    
class imgnet_testset(Dataset):
    def __init__(self, df, path="./ImageNet_darknet_dataset/data/images"):
        self.transform = transforms.PILToTensor()
        self.df = df
        self.path = path
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(os.path.join(self.path, row["filename"].split("_")[0], row["filename"]))
        return (self.transform(img).float()/255, row["label"])
        

In [9]:
def create_dataset(path="./ImageNet_darknet_dataset"):
    # in trainset, occurrences of classes goes from a max of 1200 to a minimum of 632, mean: 499
    df_main = pd.read_csv(os.path.join(path, "main.csv"), sep=";", skipinitialspace=True)
    testset_df = pd.DataFrame(columns=["filename", "label"])
    trainset_df = pd.DataFrame(columns=["filename", "label"])
    for idx in range(len(df_main)):
        print(f"CLASS {idx+1}/1000")
        row = df_main.iloc[idx]
        path_class_csv = os.path.join(path, "data", "info_classes", f"class_{row['label']}.csv")
        df_cls = pd.read_csv(path_class_csv, sep=";", skipinitialspace=True)
        # for trainset sample 100 images for each class, trainset size is 100k
        df_testset_sample = df_cls.sample(100)
        df_cls = df_cls.drop(list(df_testset_sample.index.values))
        df_cls = df_cls.sample(500)
        # add splitted dataframes to trainset and testset dataframes
        testset_df = pd.concat([testset_df, df_testset_sample], axis=0)
        trainset_df = pd.concat([trainset_df, df_cls], axis=0)
    return imgnet_trainset(trainset_df), imgnet_testset(testset_df)

# Training

In [10]:
# data augmentation
augmentation = A.Compose(
    [
        A.ColorJitter(brightness=0.6, contrast=0.6, saturation=0.6, hue=0.6, p=0.4),
        A.Blur(p=0.1),
        A.CLAHE(p=0.1),
        A.Posterize(p=0.1),
        A.ToGray(p=0.1),
        A.ChannelShuffle(p=0.05),

    ],
)

In [11]:
# perform agumentation when loading batch
def collate_fn_padd(batch):
    X = None
    Y = None
    for img, y in batch:
        img_numpy = (torch.permute(img, (1, 2, 0))*255).numpy().astype(np.uint8)
        img_numpy = augmentation(image=img_numpy)["image"]
        tensor_augm = torch.permute(torch.from_numpy(img_numpy), (2, 0, 1)) / 255
        if X == None:
            X = torch.stack((img, tensor_augm))
            Y = torch.cat((y, y))
        else:
            X = torch.cat((X, img.unsqueeze(0), tensor_augm.unsqueeze(0)))
            Y = torch.cat((Y, y, y))
    return (X, Y)

In [8]:
classes = 1000
lr = 1e-3

In [ ]:
trainset, testset = create_dataset()

In [ ]:
darknet = darknet53(darknet_architecture, classes).to(device)
ce_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=darknet.parameters(), lr=lr)
i=1

best_parameters = None
best_loss = math.inf
while(True):
    losses = []
    n_batch = 0
    train_loader = DataLoader(trainset, batch_size=8, shuffle=True)
    total_batches = len(train_loader)
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        pred = darknet(x)
        loss = ce_loss(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if n_batch % 1000 == 0:
            print(f"BATCH: {n_batch+1}/{total_batches}")
            print(f"loss: {loss.item()}\n")
        n_batch+=1
        losses.append(loss.item())
        
    mean_losses = sum(losses)/len(losses)
    if mean_losses < best_loss:
        if best_loss != math.inf:
            os.remove(f"../backup_models/darknet_weights_loss_{best_loss}")
        best_loss = mean_losses
        best_parameters = darknet.state_dict()
        torch.save(best_parameters, f"../backup_models/darknet_weights_loss_{best_loss}")
    print(f"EPOCH: {i} MEAN LOSSES EPOCH: {sum(losses)/len(losses)}")
    print(f"Loss: {loss.item()}")
    i+=1

In [ ]:
N = len(testset)
print(f"N :{N}")
top1 = 0
top5 = 0
i=1
with torch.no_grad():
    batch_size = 8
    test_loader = DataLoader(testset, batch_size=batch_size)
    N_test_loader = len(test_loader)
    for x, y in test_loader:

        x = x.to(device)
        y = y.to(device)

        pred = darknet(x)
        
        for idx_batch in range(x.shape[0]):
            top1_pred = torch.argmax(pred[idx_batch])
            if top1_pred == y[idx_batch]:
                top1 += 1
                top5 += 1
                continue
            
            top5_pred = torch.topk(pred[idx_batch], 5).indices
            if y[idx_batch] in top5_pred:
                top5 += 1
        if i%1000 == 0:
            print(f"{i}/{N_test_loader}")
        i+=1
        
print(f"ACCURACY TOP1: {top1/N}")
print(f"ACCURACY TOP5: {top5/N}")